# Prepare WDPA Input Data


### Import libraries and set up environment

In [1]:
import os
import sys

import pandas as pd
print(pd.__version__)

import numpy as np
print(np.__version__)

import arcpy

import arcgis
print(arcgis.__version__)

from arcgis.gis import GIS
from arcgis.mapping import WebMap
from arcgis.features import FeatureLayer

1.4.4
1.20.1
2.1.0.2


In [2]:
# Set the workspace and environment settings

arcpy.env.workspace = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"
arcpy.env.overwriteOutput = True 

In [ ]:
# Make sure Carib_mol_30km is in the project (made in EC_study_area_prep)
# Make sure Caribbean_detailed_subregions_mol is in the project (made in EC_study_area_prep)

In [4]:
#Add the polygon and point September 2023 WDPA data from the O drive to the ArcGIS project

output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"
arcpy.conversion.FeatureClassToGeodatabase(['O:/f00_data/Protected_Planet_Initiative/WDPA_Monthly_Versions/Previous WDPA Versions/2023/September_2023/WDPA_Sep2023_Public.gdb/WDPA_poly_Sep2023', 'O:/f00_data/Protected_Planet_Initiative/WDPA_Monthly_Versions/Previous WDPA Versions/2023/September_2023/WDPA_Sep2023_Public.gdb/WDPA_point_Sep2023'],
                                           output_gdb)

<Result 'F:\\Bex\\ArcGIS\\Ecological_coherence_2023\\Ecological_coherence_2023.gdb'>

In [14]:
# Buffer points of the WDPA to be size of Rep_Area

points = "WDPA_point_Sep2023"
out_feature = "\\WDPA_pt_Mol_buf"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

def buffer_points(points):
    #Project to Mollweide (WDPA_pt_Mol)
    arcpy.management.Project(points, "WDPA_pt_Mol", 'PROJCS["World_Mollweide",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]', None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")
    
    #Calculate radius of buffer on WDPA_pt_Mol layer
    #math.sqrt(!REP_AREA!/math.pi)*1000
    arcpy.management.CalculateField("WDPA_pt_Mol", "radius", "math.sqrt(!REP_AREA!/math.pi)*1000", "PYTHON3")

    #buffer to radius.
    #points without area were automatically removed
    arcpy.analysis.Buffer("WDPA_pt_Mol", output_gdb + out_feature, "radius", "FULL", "ROUND", "NONE", None, "PLANAR")

    #checked area to make sure it matched (field called area. calc in mollweide, area in sq km)
    arcpy.management.CalculateGeometryAttributes(out_feature, "area AREA", '', "SQUARE_KILOMETERS", 'PROJCS["World_Mollweide",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]', "SAME_AS_INPUT")

buffer_points(points)

In [15]:
# Append the point layer to the polygon layer

buffered_points = "WDPA_pt_Mol_buf"
poly = "WDPA_poly_Sep2023"

def append_points_to_poly():
    #Project to Mollweide
    arcpy.management.Project(poly, output_gdb + "\\WDPA_poly_Mol", 'PROJCS["World_Mollweide",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]', None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")

    #repair geometry
    arcpy.management.RepairGeometry("WDPA_poly_Mol", "DELETE_NULL", "OGC")
    
    #rename it as WDPA_all to get ready to append it with points
    arcpy.conversion.ExportFeatures("WDPA_poly_Mol", output_gdb + "\\WDPA_all")
    
    #Append the WDPA_all with the point layer (WDPA_pt_Mol_buf). 
    #Note that append worked better than merge because merge didn't keep the attributes of the points
    arcpy.management.Append(buffered_points, "WDPA_all", "NO_TEST")
    
    #repair geometry
    arcpy.management.RepairGeometry("WDPA_all", "DELETE_NULL", "OGC")

append_points_to_poly()

In [16]:
# Exclude sites not used in the analysis
# Exclude all sites that are proposed or not-reported.
# Remove MABs

in_feature = "WDPA_all"
out_feature = "\\WDPA"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

def remove_sites():
    #first remove sites with status proposed or not-reported.
    arcpy.conversion.ExportFeatures(in_feature, output_gdb + "\\WDPA_RemovePropNR", "STATUS <> 'Proposed' AND STATUS <> 'Not Reported'" )

    #Then remove PAs that are MABs
    arcpy.conversion.ExportFeatures("WDPA_RemovePropNR", output_gdb + out_feature, "DESIG_ENG NOT LIKE '%UNESCO-MAB%'" )
    
    #This final layer is called WDPA
    #Repair geometry on the WDPA layer
    arcpy.management.RepairGeometry(out_feature, "DELETE_NULL", "OGC")

remove_sites()


### Make the habitat MPA layer

In [12]:
# Clip the WDPA layer to the study area buffered by 30 km
# This will just be used for the habitat analysis

in_feature = "WDPA"
clip_feature = "Carib_mol_30km"
out_feature = "\\WDPA_30km"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

arcpy.analysis.Clip(in_feature, clip_feature, out_feature)

<Result 'F:\\Bex\\ArcGIS\\Ecological_coherence_2023\\Ecological_coherence_2023.gdb\\\\WDPA_30km'>

In [13]:
# Dissolve the WDPA_30km for a flat layer
# Make them single part for faster processing
# this will just be used for the habitat analysis

in_feature = "WDPA_30km"
out_feature = "\\WDPA_30km_dis"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

arcpy.analysis.PairwiseDissolve(
    in_features= in_feature,
    out_feature_class= output_gdb + out_feature,
    dissolve_field=None,
    statistics_fields=None,
    multi_part="SINGLE_PART",
    concatenation_separator=""
)


<Result 'F:\\Bex\\ArcGIS\\Ecological_coherence_2023\\Ecological_coherence_2023.gdb\\WDPA_30km_dis'>

### Make the main MPA layer

In [4]:
# Select the PAs that intersect with the study area.

in_feature = "WDPA"
select_feature = "Caribbean_detailed_subregions_mol"
out_feature = "\\WDPA_Carib_int"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

def intersecting_PAs():
    # Select PAs that intersect with the study area
    selection = arcpy.management.SelectLayerByLocation(
        in_layer= in_feature,
        overlap_type="INTERSECT",
        select_features= select_feature,
        search_distance=None,
        selection_type="NEW_SELECTION",
        invert_spatial_relationship="NOT_INVERT"
    )
        
    # make a new layer from the selection
    arcpy.conversion.ExportFeatures(
        in_features= selection,
        out_features= output_gdb + out_feature,
        where_clause="",
        use_field_alias_as_name="NOT_USE_ALIAS",
        sort_field=None
    )
    #arcpy.conversion.ExportFeatures(in_feature, output_gdb + out_feature)
    
intersecting_PAs()


In [5]:
# Clip the WDPA_Carib_int to just the coast

in_feature = "WDPA_Carib_int"
clip_feature = "Caribbean_detailed_subregions_mol"
out_feature = "\\WDPA_Carib_CoastClip"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

arcpy.analysis.Clip(in_feature, clip_feature, out_feature)



<Result 'F:\\Bex\\ArcGIS\\Ecological_coherence_2023\\Ecological_coherence_2023.gdb\\\\WDPA_Carib_CoastClip'>

In [6]:
# Calculate the area (in hectares) of the PAs in WDPA_Carib_CoastClip and in WDPA_Carib_int
# insert "area" field (double) to all six intersections and calculate area in km2 in Mollweide

arcpy.management.CalculateGeometryAttributes(
    in_features= "WDPA_Carib_CoastClip",
    geometry_property="marine_area_ha AREA",
    length_unit="",
    area_unit="HECTARES",
    coordinate_system='PROJCS["World_Mollweide",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]',
    coordinate_format="SAME_AS_INPUT"
)

arcpy.management.CalculateGeometryAttributes(
    in_features= "WDPA_Carib_int",
    geometry_property="total_area_ha AREA",
    length_unit="",
    area_unit="HECTARES",
    coordinate_system='PROJCS["World_Mollweide",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]',
    coordinate_format="SAME_AS_INPUT"
)

<Result 'F:\\Bex\\ArcGIS\\Ecological_coherence_2023\\Ecological_coherence_2023.gdb\\WDPA_Carib_int'>

In [ ]:
# Join the new areas field in WDPA_Carib_int to the WDPA_Carib_CoastClip layer.
# Join based on WDPA_PID
# Using field mapping, only keep the "total_area_ha" field from the join table

arcpy.management.JoinField(
    in_data="WDPA_Carib_CoastClip",
    in_field="WDPA_PID",
    join_table="WDPA_Carib_int",
    join_field="WDPA_PID",
    fields=None,
    fm_option="USE_FM",
    field_mapping='total_area_ha "total_area_ha" true true false 8 Double 0 0,First,#,WDPA_Carib_int,total_area_ha,-1,-1'
)

In [8]:
# Make join permenant by exporting the data

in_feature = "WDPA_Carib_CoastClip"
out_feature = "\\WDPA_Carib_CoastClip_join"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

arcpy.conversion.ExportFeatures(
    in_features= in_feature,
    out_features= output_gdb + out_feature,
    where_clause="",
    use_field_alias_as_name="NOT_USE_ALIAS",
    sort_field=None
)

<Result 'F:\\Bex\\ArcGIS\\Ecological_coherence_2023\\Ecological_coherence_2023.gdb\\WDPA_Carib_CoastClip_join'>

In [ ]:
# Add a new field in WDPA_Carib_CoastClip_join to Calculate percent marine area

arcpy.management.CalculateField(
    in_table="WDPA_Carib_CoastClip_join",
    field="Percent_marine_area",
    expression="!marine_area_ha! / !total_area_ha! * 100",
    expression_type="PYTHON3",
    code_block="",
    field_type="DOUBLE",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

Note that I wasn't sure how to deal with PAs with multiple parcels in terms of deciding if parcels should be considered together or seperately when determining if they are MPAs. But I checked, and there are only two PAs with multiple parcels, and in both cases, either way they would be considered MPAs.

In [ ]:
# Add a field to determine if the polygon should be considered an MPA
# The clipped marine portion of coast-overlapping sites were identified as MPAs in this assessment if
# more than 100 hectares of the site fell in the marine environment,
# or more than 10 hectares of the site fell in the marine environment and this represented more than 30% of the site

arcpy.management.CalculateField(
    in_table="WDPA_Carib_CoastClip_join",
    field="MPA",
    expression="mpa(!marine_area_ha!, !Percent_marine_area!)",
    expression_type="PYTHON3",
    code_block="""def mpa(marine_area_ha, Percent_marine_area):
    if marine_area_ha >= 100:
        return "yes"
    elif marine_area_ha > 10 and Percent_marine_area >30:
        return "yes"
    else:
        return "no"
""",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

In [11]:
# Make a layer of the PAs that are considered MPAs

in_feature = "WDPA_Carib_CoastClip_join"
out_feature = "\\WDPA_MPA"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

arcpy.conversion.ExportFeatures(
    in_features= in_feature,
    out_features= output_gdb + out_feature,
    where_clause="MPA = 'yes'",
    use_field_alias_as_name="NOT_USE_ALIAS",
    sort_field=None
)

<Result 'F:\\Bex\\ArcGIS\\Ecological_coherence_2023\\Ecological_coherence_2023.gdb\\WDPA_MPA'>

In [ ]:
# Dissolve the MPAs for a flat layer
# Make them single part for faster processing

in_feature = "WDPA_MPA"
out_feature = "\\WDPA_MPA_dis"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

arcpy.analysis.PairwiseDissolve(
    in_features= in_feature,
    out_feature_class= output_gdb + out_feature,
    dissolve_field=None,
    statistics_fields=None,
    multi_part="SINGLE_PART",
    concatenation_separator=""
)


### Make the no-take MPA layer

In [4]:
# Start with the WDPA_MPA layer

# Only keep features where NO_TAKE == all or part

in_feature = "WDPA_MPA"
out_feature = "\\WDPA_MPA_NoTake"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

arcpy.conversion.ExportFeatures(
    in_features=in_feature,
    out_features=output_gdb + out_feature,
    where_clause="NO_TAKE = 'All' Or NO_TAKE = 'Part'",
    use_field_alias_as_name="NOT_USE_ALIAS",
    field_mapping='WDPAID "WDPAID" true true false 8 Double 0 0,First,#,WDPA_MPA,WDPAID,-1,-1;WDPA_PID "WDPA_PID" true true false 52 Text 0 0,First,#,WDPA_MPA,WDPA_PID,0,52;PA_DEF "PA_DEF" true true false 20 Text 0 0,First,#,WDPA_MPA,PA_DEF,0,20;NAME "NAME" true true false 254 Text 0 0,First,#,WDPA_MPA,NAME,0,254;ORIG_NAME "ORIG_NAME" true true false 254 Text 0 0,First,#,WDPA_MPA,ORIG_NAME,0,254;DESIG "DESIG" true true false 254 Text 0 0,First,#,WDPA_MPA,DESIG,0,254;DESIG_ENG "DESIG_ENG" true true false 254 Text 0 0,First,#,WDPA_MPA,DESIG_ENG,0,254;DESIG_TYPE "DESIG_TYPE" true true false 20 Text 0 0,First,#,WDPA_MPA,DESIG_TYPE,0,20;IUCN_CAT "IUCN_CAT" true true false 20 Text 0 0,First,#,WDPA_MPA,IUCN_CAT,0,20;INT_CRIT "INT_CRIT" true true false 100 Text 0 0,First,#,WDPA_MPA,INT_CRIT,0,100;MARINE "MARINE" true true false 20 Text 0 0,First,#,WDPA_MPA,MARINE,0,20;REP_M_AREA "REP_M_AREA" true true false 8 Double 0 0,First,#,WDPA_MPA,REP_M_AREA,-1,-1;GIS_M_AREA "GIS_M_AREA" true true false 8 Double 0 0,First,#,WDPA_MPA,GIS_M_AREA,-1,-1;REP_AREA "REP_AREA" true true false 8 Double 0 0,First,#,WDPA_MPA,REP_AREA,-1,-1;GIS_AREA "GIS_AREA" true true false 8 Double 0 0,First,#,WDPA_MPA,GIS_AREA,-1,-1;NO_TAKE "NO_TAKE" true true false 50 Text 0 0,First,#,WDPA_MPA,NO_TAKE,0,50;NO_TK_AREA "NO_TK_AREA" true true false 8 Double 0 0,First,#,WDPA_MPA,NO_TK_AREA,-1,-1;STATUS "STATUS" true true false 100 Text 0 0,First,#,WDPA_MPA,STATUS,0,100;STATUS_YR "STATUS_YR" true true false 4 Long 0 0,First,#,WDPA_MPA,STATUS_YR,-1,-1;GOV_TYPE "GOV_TYPE" true true false 254 Text 0 0,First,#,WDPA_MPA,GOV_TYPE,0,254;OWN_TYPE "OWN_TYPE" true true false 254 Text 0 0,First,#,WDPA_MPA,OWN_TYPE,0,254;MANG_AUTH "MANG_AUTH" true true false 254 Text 0 0,First,#,WDPA_MPA,MANG_AUTH,0,254;MANG_PLAN "MANG_PLAN" true true false 254 Text 0 0,First,#,WDPA_MPA,MANG_PLAN,0,254;VERIF "VERIF" true true false 20 Text 0 0,First,#,WDPA_MPA,VERIF,0,20;METADATAID "METADATAID" true true false 4 Long 0 0,First,#,WDPA_MPA,METADATAID,-1,-1;SUB_LOC "SUB_LOC" true true false 100 Text 0 0,First,#,WDPA_MPA,SUB_LOC,0,100;PARENT_ISO3 "PARENT_ISO3" true true false 50 Text 0 0,First,#,WDPA_MPA,PARENT_ISO3,0,50;ISO3 "ISO3" true true false 50 Text 0 0,First,#,WDPA_MPA,ISO3,0,50;marine_area_ha "marine_area_ha" true true false 8 Double 0 0,First,#,WDPA_MPA,marine_area_ha,-1,-1;total_area_ha "total_area_ha" true true false 8 Double 0 0,First,#,WDPA_MPA,total_area_ha,-1,-1;Percent_marine_area "Percent_marine_area" true true false 8 Double 0 0,First,#,WDPA_MPA,Percent_marine_area,-1,-1;MPA "MPA" true true false 512 Text 0 0,First,#,WDPA_MPA,MPA,0,512;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,WDPA_MPA,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,WDPA_MPA,Shape_Area,-1,-1',
    sort_field=None
)

<Result 'F:\\Bex\\ArcGIS\\Ecological_coherence_2023\\Ecological_coherence_2023.gdb\\WDPA_MPA_NoTake'>

In [5]:
# Dissolve the no-take MPAs for a flat layer
# Make them single part for faster processing

in_feature = "WDPA_MPA_NoTake"
out_feature = "\\WDPA_MPA_NoTake_dis"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

arcpy.analysis.PairwiseDissolve(
    in_features= in_feature,
    out_feature_class= output_gdb + out_feature,
    dissolve_field=None,
    statistics_fields=None,
    multi_part="SINGLE_PART",
    concatenation_separator=""
)


<Result 'F:\\Bex\\ArcGIS\\Ecological_coherence_2023\\Ecological_coherence_2023.gdb\\WDPA_MPA_NoTake_dis'>

### Make the no-take MPA HABITAT layer

In [3]:
# start with the undissolved habitat MPA layer: "WDPA_30km"

# Only keep features where NO_TAKE == all or part

in_feature = "WDPA_30km"
out_feature = "\\WDPA_MPA_30km_NoTake"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

arcpy.conversion.ExportFeatures(
    in_features=in_feature,
    out_features=output_gdb + out_feature,
    where_clause="NO_TAKE = 'All' Or NO_TAKE = 'Part'",
    use_field_alias_as_name="NOT_USE_ALIAS",
    field_mapping=None, 
    sort_field=None
)



<Result 'F:\\Bex\\ArcGIS\\Ecological_coherence_2023\\Ecological_coherence_2023.gdb\\WDPA_MPA_30km_NoTake'>

In [4]:
# Dissolve the no-take MPA habitat layer for a flat layer
# Make them single part for faster processing

in_feature = "WDPA_MPA_30km_NoTake"
out_feature = "\\WDPA_MPA_30km_NoTake_dis"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

arcpy.analysis.PairwiseDissolve(
    in_features= in_feature,
    out_feature_class= output_gdb + out_feature,
    dissolve_field=None,
    statistics_fields=None,
    multi_part="SINGLE_PART",
    concatenation_separator=""
)

<Result 'F:\\Bex\\ArcGIS\\Ecological_coherence_2023\\Ecological_coherence_2023.gdb\\WDPA_MPA_30km_NoTake_dis'>